<a href="https://colab.research.google.com/github/GiovanniPasq/agentic-rag-for-dummies/blob/main/Agentic_Rag_For_Dummies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#run this cell only in colab, otherwise create a venv and install requirements.txt available in the project folder
!pip install --quiet --upgrade langgraph
!pip install -qU "langchain[google-genai]"
!pip install -qU langchain langchain-community langchain-qdrant langchain-huggingface qdrant-client fastembed flashrank langchain-core
!pip install --upgrade gradio

# Optional: if you want to use Ollama with local models
!pip install -qU langchain-ollama

In [ ]:
# ============================================================
# BLOCK 1: Configuration and Environment Setup
# ============================================================
import os
from pathlib import Path

# Configuration
DOCS_DIR = "docs"  # Directory containing your .md files
PARENT_STORE_PATH = "parent_store"  # Directory for parent chunk JSON files
CHILD_COLLECTION = "document_child_chunks"

# Create directories if they don't exist
os.makedirs(DOCS_DIR, exist_ok=True)
os.makedirs(PARENT_STORE_PATH, exist_ok=True)

print("✓ Configuration loaded")
print(f"  - Docs directory: {DOCS_DIR}")
print(f"  - Parent store: {PARENT_STORE_PATH}")
print(f"  - Collection name: {CHILD_COLLECTION}")

✓ Configuration loaded
  - Docs directory: docs
  - Parent store: parent_store
  - Collection name: document_child_chunks


In [ ]:
# ============================================================
# BLOCK 2: LLM Initialization
# ============================================================
from langchain_ollama import ChatOllama

# Initialize LLM
llm = ChatOllama(model="qwen3:4b-instruct-2507-q4_K_M", temperature=0.1)

# Alternative: Google Gemini
# from langchain_google_genai import ChatGoogleGenerativeAI
# os.environ["GOOGLE_API_KEY"] = "your-api-key-here"
# llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0)

print("✓ LLM initialized")

In [ ]:
# ============================================================
# BLOCK 3: Embeddings Setup
# ============================================================
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant.fastembed_sparse import FastEmbedSparse

# Dense embeddings for semantic understanding
dense_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

# Sparse embeddings for keyword matching
sparse_embeddings = FastEmbedSparse(
    model_name="Qdrant/bm25"
)

print("✓ Embeddings initialized")
print(f"  - Dense model: sentence-transformers/all-mpnet-base-v2")
print(f"  - Sparse model: Qdrant/bm25")

✓ Embeddings initialized
  - Dense model: sentence-transformers/all-mpnet-base-v2
  - Sparse model: Qdrant/bm25


In [ ]:
# ============================================================
# BLOCK 4: Vector Database Setup
# ============================================================
from qdrant_client import QdrantClient
from qdrant_client.http import models as qmodels
from langchain_qdrant import QdrantVectorStore
from langchain_qdrant.qdrant import RetrievalMode

# Initialize Qdrant client (local file-based storage)
client = QdrantClient(path="qdrant_db")

# Get embedding dimension
embedding_dimension = len(dense_embeddings.embed_query("test"))

def ensure_collection(collection_name):
    """Create Qdrant collection if it doesn't exist"""
    if not client.collection_exists(collection_name):
        client.create_collection(
            collection_name=collection_name,
            vectors_config=qmodels.VectorParams(
                size=embedding_dimension,
                distance=qmodels.Distance.COSINE
            ),
            sparse_vectors_config={
                "sparse": qmodels.SparseVectorParams()
            },
        )
        print(f"✓ Created collection: {collection_name}")
    else:
        print(f"✓ Collection already exists: {collection_name}")

# Create collection
ensure_collection(CHILD_COLLECTION)

# Initialize vector store for child chunks
child_vector_store = QdrantVectorStore(
    client=client,
    collection_name=CHILD_COLLECTION,
    embedding=dense_embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    sparse_vector_name="sparse"
)

print("✓ Vector store initialized")

✓ Created collection: document_child_chunks
✓ Vector store initialized


In [ ]:
# ============================================================
# BLOCK 5: Document Indexing
# ============================================================
import glob
import json
from langchain_text_splitters import (
    MarkdownHeaderTextSplitter,
    RecursiveCharacterTextSplitter
)

def index_documents():
    """Index documents using hierarchical Parent/Child strategy"""
    print("\n" + "="*50)
    print("Starting Hierarchical Indexing")
    print("="*50 + "\n")

    # Parent splitter: by Markdown headers
    headers_to_split_on = [
        ("#", "H1"),
        ("##", "H2"),
        ("###", "H3")
    ]
    parent_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on,
        strip_headers=False
    )

    # Child splitter: by character count
    child_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100
    )

    all_child_chunks = []
    all_parent_pairs = []

    # Check if docs directory has files
    md_files = sorted(glob.glob(os.path.join(DOCS_DIR, "*.md")))
    if not md_files:
        print(f"⚠️  No .md files found in {DOCS_DIR}/")
        print("Please add your Markdown documents to continue.")
        return

    # Process each document
    for doc_path_str in md_files:
        doc_path = Path(doc_path_str)
        print(f"📄 Processing: {doc_path.name}")

        try:
            with open(doc_path, "r", encoding="utf-8") as f:
                md_text = f.read()
        except Exception as e:
            print(f"❌ Error reading {doc_path.name}: {e}")
            continue

        # Split into parent chunks
        parent_chunks = parent_splitter.split_text(md_text)

        for i, p_chunk in enumerate(parent_chunks):
            # Add metadata
            p_chunk.metadata["source"] = str(doc_path)
            parent_id = f"{doc_path.stem}_parent_{i}"
            p_chunk.metadata["parent_id"] = parent_id

            # Store parent reference
            all_parent_pairs.append((parent_id, p_chunk))

            # Split into child chunks
            child_chunks = child_splitter.split_documents([p_chunk])
            all_child_chunks.extend(child_chunks)

    # Save child chunks to Qdrant
    if all_child_chunks:
        print(f"\n🔍 Indexing {len(all_child_chunks)} child chunks into Qdrant...")
        try:
            child_vector_store.add_documents(all_child_chunks)
            print("✓ Child chunks indexed successfully")
        except Exception as e:
            print(f"❌ Error indexing child chunks: {e}")
            return
    else:
        print("⚠️  No child chunks to index")
        return

    # Save parent chunks to JSON files
    if all_parent_pairs:
        print(f"💾 Saving {len(all_parent_pairs)} parent chunks to JSON...")

        # Clear existing parent files
        for item in os.listdir(PARENT_STORE_PATH):
            os.remove(os.path.join(PARENT_STORE_PATH, item))

        # Save each parent chunk
        for parent_id, doc in all_parent_pairs:
            doc_dict = {
                "page_content": doc.page_content,
                "metadata": doc.metadata
            }
            file_path = os.path.join(PARENT_STORE_PATH, f"{parent_id}.json")
            with open(file_path, "w", encoding="utf-8") as f:
                json.dump(doc_dict, f, ensure_ascii=False, indent=2)

        print("✓ Parent chunks saved successfully")

    print("\n" + "="*50)
    print("✓ Indexing Complete!")
    print("="*50 + "\n")

# Run indexing
index_documents()


Starting Hierarchical Indexing

📄 Processing: blockchain.md
📄 Processing: fortinet.md
📄 Processing: javascript_tutorial.md
📄 Processing: microservices.md

🔍 Indexing 2447 child chunks into Qdrant...
✓ Child chunks indexed successfully
💾 Saving 386 parent chunks to JSON...
✓ Parent chunks saved successfully

✓ Indexing Complete!



In [ ]:
# ============================================================
# BLOCK 6: Tool Definitions
# ============================================================
import json
from typing import List
from langchain_core.tools import tool

@tool
def search_child_chunks(query: str, k: int = 5) -> List[dict]:
    """
    Search for the top K most relevant child chunks.

    Args:
        query: Search query string
        k: Number of results to return

    Returns:
        List of dicts with content, parent_id, and source
    """
    try:
        results = child_vector_store.similarity_search(query, k=k)
        return [
            {
                "content": doc.page_content,
                "parent_id": doc.metadata.get("parent_id", ""),
                "source": doc.metadata.get("source", "")
            }
            for doc in results
        ]
    except Exception as e:
        print(f"Error searching child chunks: {e}")
        return []

@tool
def retrieve_parent_chunks(parent_ids: List[str]) -> List[dict]:
    """
    Retrieve full parent chunks by their IDs.

    Args:
        parent_ids: List of parent chunk IDs to retrieve

    Returns:
        List of dicts with content, parent_id, and metadata
    """
    unique_ids = sorted(list(set(parent_ids)))
    results = []

    for parent_id in unique_ids:
        file_path = os.path.join(PARENT_STORE_PATH, parent_id if parent_id.lower().endswith(".json") else f"{parent_id}.json")
        if os.path.exists(file_path):
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    doc_dict = json.load(f)
                    results.append({
                        "content": doc_dict["page_content"],
                        "parent_id": parent_id,
                        "metadata": doc_dict["metadata"]
                    })
            except Exception as e:
                print(f"Error loading parent chunk {parent_id}: {e}")

    return results

# Bind tools to LLM
llm_with_tools = llm.bind_tools([search_child_chunks, retrieve_parent_chunks])

print("✓ Tools defined and bound to LLM")

✓ Tools defined and bound to LLM


In [ ]:
# ============================================================
# BLOCK 7: Agent System Prompt
# ============================================================
from langchain_core.messages import SystemMessage

AGENT_SYSTEM_PROMPT = """
You are an intelligent assistant that MUST use the available tools to answer questions.

**MANDATORY WORKFLOW - Follow these steps for EVERY question:**

1. **ALWAYS start by calling `search_child_chunks`** with the user's query
   - Choose appropriate K value (default: 5)

2. **Read the retrieved chunks** and check if they answer the question

3. **If chunks are incomplete**, call `retrieve_parent_chunks` with parent_id values

4. **Answer using ONLY the retrieved information**
   - Cite source files from metadata

5. **If no relevant information found after searching**, try rephrasing the query once more

**CRITICAL**: You MUST call tools before answering. Never respond without first using `search_child_chunks`.
"""

agent_system_message = SystemMessage(content=AGENT_SYSTEM_PROMPT)

print("✓ Agent system prompt configured")

✓ Agent system prompt configured


In [ ]:
# ============================================================
# BLOCK 8: State Definitions
# ============================================================
from langgraph.graph import MessagesState
from pydantic import BaseModel
from typing import List

class State(MessagesState):
    questionIsClear: bool
    conversation_summary: str = ""

class QueryAnalysis(BaseModel):
    is_clear: bool = Field(
        description="Indicates if the user's question is clear and answerable."
    )
    questions: List[str] = Field(
        description="List of rewritten, self-contained questions."
    )
    clarification_needed: str = Field(
        description="Explanation if the question is unclear."
    )

print("✓ State models defined")

✓ State models defined


In [ ]:
# ============================================================
# BLOCK 9: Graph Node Functions
# ============================================================
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, RemoveMessage
from typing import Literal

def analyze_chat_and_summarize(state: State):
    """
    Analyzes chat history and summarizes key points for context.
    """
    if len(state["messages"]) < 4:  # Need some history to summarize
        return {"conversation_summary": ""}

    # Extract relevant messages (excluding current query and system messages)
    relevant_msgs = [
        msg for msg in state["messages"][:-1]  # Exclude current query
        if isinstance(msg, (HumanMessage, AIMessage))
        and not getattr(msg, "tool_calls", None)
    ]

    if not relevant_msgs:
        return {"conversation_summary": ""}

    summary_prompt = "Summarize the key topics and context from this conversation concisely (2-3 sentences max). Discard irrelevant information, such as misunderstandings or off-topic queries/responses. If there are no key topics, return an empty string:\n\n"
    for msg in relevant_msgs[-6:]:  # Last 6 messages for context
        role = "User" if isinstance(msg, HumanMessage) else "Assistant"
        summary_prompt += f"{role}: {msg.content}\n"

    summary_prompt += "\nBrief Summary:"
    summary_response = llm.with_config(temperature=0.2).invoke([SystemMessage(content=summary_prompt)])
    return {"conversation_summary": summary_response.content}

def analyze_and_rewrite_query(state: State):
    """
    Analyzes user query and rewrites it for clarity, optionally using conversation context.
    """
    last_message = state["messages"][-1]
    conversation_summary = state.get("conversation_summary", "")

    context_section = (
        f"**Conversation Context:**\n{conversation_summary}"
        if conversation_summary.strip()
        else "**Conversation Context:**\n[First query in conversation]"
    )

    # Create analysis prompt
    prompt = f"""Rewrite the user's query to be clear and optimized for information retrieval.
                **User Query:**
                `"{last_message.content}"`

                `{context_section}`

                **Instructions:**

                1.  **If this is a follow-up** (uses pronouns, references previous topics): Resolve all references using the context to make it a self-contained query.
                2.  **If it's a new independent query:** Ensure it's already clear and specific.
                3.  **If it contains multiple distinct questions:** Split them into a list of separate, focused questions.
                4.  **Use specific, keyword-rich language:** Remove vague terms and conversational filler (e.g., "tell me about" -> "facts about").
                5.  **Mark as unclear** if you cannot determine a clear user intent for information retrieval.
                    * This includes queries that are **nonsense/gibberish**, **insults**, or statements without an apparent question.
                """

    llm_with_structure = llm.with_config(temperature=0.2).with_structured_output(QueryAnalysis)
    response = llm_with_structure.invoke([SystemMessage(content=prompt)])

    if response.is_clear:
        # Remove all non-system messages
        delete_all = [
            RemoveMessage(id=m.id)
            for m in state["messages"]
            if not isinstance(m, SystemMessage)
        ]

        # Format rewritten query
        rewritten = (
            "\n".join([f"{i+1}. {q}" for i, q in enumerate(response.questions)])
            if len(response.questions) > 1
            else response.questions[0]
        )
        return {
            "questionIsClear": True,
            "messages": delete_all + [HumanMessage(content=rewritten)]
        }
    else:
        clarification = response.clarification_needed or "I need more information to understand your question."
        return {
            "questionIsClear": False,
            "messages": [AIMessage(content=clarification)]
        }

def human_input_node(state: State):
    """Placeholder node for human-in-the-loop interruption"""
    return {}

def route_after_rewrite(state: State) -> Literal["agent", "human_input"]:
    """Route to agent if question is clear, otherwise wait for human input"""
    return "agent" if state.get("questionIsClear", False) else "human_input"

def agent_node(state: State):
    """Main agent node that processes queries using tools"""
    messages = [SystemMessage(content=agent_system_message.content)] + state["messages"]
    response = llm_with_tools.with_config(temperature=0.1).invoke(messages)
    return {"messages": [response]}

print("✓ Graph node functions defined")

✓ Graph node functions defined


In [ ]:
# ============================================================
# BLOCK 10: Graph Construction
# ============================================================
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import InMemorySaver

# Initialize checkpointer
checkpointer = InMemorySaver()

# Create graph builder
graph_builder = StateGraph(State)

# Add nodes
graph_builder.add_node("summarize", analyze_chat_and_summarize)
graph_builder.add_node("analyze_rewrite", analyze_and_rewrite_query)
graph_builder.add_node("human_input", human_input_node)
graph_builder.add_node("agent", agent_node)
graph_builder.add_node("tools", ToolNode([search_child_chunks, retrieve_parent_chunks]))

# Add edges
graph_builder.add_edge(START, "summarize")
graph_builder.add_edge("summarize", "analyze_rewrite")
graph_builder.add_conditional_edges("analyze_rewrite", route_after_rewrite)
graph_builder.add_edge("human_input", "analyze_rewrite")
graph_builder.add_conditional_edges("agent", tools_condition)
graph_builder.add_edge("tools", "agent")

# Compile graph
agent_graph = graph_builder.compile(
    checkpointer=checkpointer,
    interrupt_before=["human_input"]
)

print("✓ LangGraph agent compiled")
print("  - Nodes: summarize, analyze_rewrite, human_input, agent, tools")
print("  - Human-in-the-loop enabled at: human_input")

✓ LangGraph agent compiled
  - Nodes: summarize, analyze_rewrite, human_input, agent, tools
  - Human-in-the-loop enabled at: human_input


In [ ]:
# ============================================================
# BLOCK 11: Gradio Interface
# ============================================================
import gradio as gr
import uuid

def create_thread_id():
    """Generate a unique thread ID for each conversation"""
    return {"configurable": {"thread_id": str(uuid.uuid4())}}

def clear_session():
    """Clear thread for new conversation and clean up checkpointer state"""
    agent_graph.checkpointer.delete_thread(config["configurable"]["thread_id"])
    config = create_thread_id()

def chat_with_agent(message, history):
    """
    Handle chat with human-in-the-loop support.
    Returns: response text
    """
    current_state = agent_graph.get_state(config)
    if current_state.next:
        agent_graph.update_state(config,{"messages": [HumanMessage(content=message.strip())]})
        result = agent_graph.invoke(None, config)
    else:
        result = agent_graph.invoke({"messages": [HumanMessage(content=message.strip())]}, config)
    return result['messages'][-1].content

# Initialize thread configuration
config = create_thread_id()

# Create Gradio interface
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=600, placeholder="<strong>Ask me anything!</strong><br><em>I'll search, reason, and act to give you the best answer :)</em>")
    chatbot.clear(clear_session)
    gr.ChatInterface(fn=chat_with_agent, type="messages", chatbot=chatbot)

print("\nLaunching application...")
demo.launch()